In [6]:
import numpy as np
from scipy.stats import nbinom
import matplotlib.pyplot as plt

a = np.array([1,2,3,4,5,6,7,8])
v = np.array([4,5,6,6,7,9,10])

for i in range(len(a)):
    print(nbinom.logpmf(a,n=10,p=0.00000001))



[-181.90422236 -180.19947427 -178.81317992 -177.63452494 -176.60490553
 -175.68861481 -174.86193624 -174.10816445]
[-181.90422236 -180.19947427 -178.81317992 -177.63452494 -176.60490553
 -175.68861481 -174.86193624 -174.10816445]
[-181.90422236 -180.19947427 -178.81317992 -177.63452494 -176.60490553
 -175.68861481 -174.86193624 -174.10816445]
[-181.90422236 -180.19947427 -178.81317992 -177.63452494 -176.60490553
 -175.68861481 -174.86193624 -174.10816445]
[-181.90422236 -180.19947427 -178.81317992 -177.63452494 -176.60490553
 -175.68861481 -174.86193624 -174.10816445]
[-181.90422236 -180.19947427 -178.81317992 -177.63452494 -176.60490553
 -175.68861481 -174.86193624 -174.10816445]
[-181.90422236 -180.19947427 -178.81317992 -177.63452494 -176.60490553
 -175.68861481 -174.86193624 -174.10816445]
[-181.90422236 -180.19947427 -178.81317992 -177.63452494 -176.60490553
 -175.68861481 -174.86193624 -174.10816445]
